In [1]:
from functools import reduce
import pandas as pd
import pprint

class Classifier():
    data = None
    class_attr = None
    priori = {}
    cp = {}
    hypothesis = None

    def __init__(self, filename=None, class_attr=None):
        self.data = pd.read_csv(filename, sep=',', header=(0))
        self.class_attr = class_attr

    def calculate_priori(self):
        class_values = list(set(self.data[self.class_attr]))
        class_data = list(self.data[self.class_attr])
        total_instances = float(len(class_data))

        for i in class_values:
            self.priori[i] = class_data.count(i) / total_instances
        print("Priori Values: ", self.priori)

    def get_cp(self, attr, attr_type, class_value):
        data_attr = list(self.data[attr])
        class_data = list(self.data[self.class_attr])
        total = 1
        for i in range(0, len(data_attr)):
            if class_data[i] == class_value and data_attr[i] == attr_type:
                total += 1
        return total / float(class_data.count(class_value) + len(set(data_attr)))

    def calculate_conditional_probabilities(self, hypothesis):
        for i in self.priori:
            self.cp[i] = {}
            for j in hypothesis:
                self.cp[i].update({hypothesis[j]: self.get_cp(j, hypothesis[j], i)})
        print("\nCalculated Conditional Probabilities: \n")
        pprint.pprint(self.cp)

    def classify(self):
        print("Result: ")
        total_prob = 0  
        for i in self.cp:
            total_prob += reduce(lambda x, y: x * y, self.cp[i].values()) * self.priori[i]

        # Imprimir los resultados normalizados
        for i in self.cp:
            prob = (reduce(lambda x, y: x * y, self.cp[i].values()) * self.priori[i]) / total_prob
            print(i, " ==> ", prob)


if __name__ == "__main__":
    c = Classifier(filename="tienda.csv", class_attr="Popularidad")
    c.calculate_priori()
    c.hypothesis = {"Categoria":'Accesorios'}

    c.calculate_conditional_probabilities(c.hypothesis)
    c.classify()

Priori Values:  {'No Popular': 0.42857142857142855, 'Popular': 0.5714285714285714}

Calculated Conditional Probabilities: 

{'No Popular': {'Accesorios': 0.625}, 'Popular': {'Accesorios': 0.3}}
Result: 
No Popular  ==>  0.6097560975609756
Popular  ==>  0.39024390243902435
